# Drone follow me using Kalman Filters

Multi-Object Tracking (MOT) is a core visual ability that humans poses to perform kinetic tasks and coordinate other tasks. The AI community has recognized the importance of MOT via a series of [competitions](https://motchallenge.net). 

In this assignment, the object class is `bicycle` and `car` the ability to track these objects  will be demonstrated using [Kalman Filters](https://en.wikipedia.org/wiki/Kalman_filter).  

## Task 1: Setup your development environment and store the test video locally (10 points)

Your environment must be docker based and you can use any TF2 or PT2 based docker container compatible with your environment. You can also use colab. 

## Task 2: Object Detection (40 points)

Perform object detection on the following videos. 

Split the videos into frames and use an object detector of your choice, in a framework of your choice to detect the cyclists.  

In [5]:
import cv2
import numpy as np
from filterpy.kalman import KalmanFilter

def initialize_kalman_filter():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 0, 1, 0], [0, 1, 0, 1], [0, 0, 1, 0], [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0], [0, 1, 0, 0]])
    kf.R *= 10
    kf.P *= 1000
    kf.Q = np.eye(kf.dim_x) * 0.01
    return kf

def select_objects(frame):
    objects = []
    print("Select objects and press ENTER or SPACE when done. Press 'c' to cancel the current selection.")
    while True:
        roi = cv2.selectROI("Frame", frame, fromCenter=False, showCrosshair=True)
        if roi == (0, 0, 0, 0):
            break
        kf = initialize_kalman_filter()
        objects.append([roi, kf, [], None])
    cv2.destroyAllWindows()
    return objects

def track_objects(frame, objects):
    for object in objects:
        roi = object[0]
        tracker = cv2.TrackerCSRT_create()
        tracker.init(frame, roi)
        object[3] = tracker

def update_trackers(objects, frame):
    for object in objects:
        tracker = object[3]
        success, box = tracker.update(frame)
        if success:
            p1 = (int(box[0]), int(box[1]))
            p2 = (int(box[0] + box[2]), int(box[1] + box[3]))
            cv2.rectangle(frame, p1, p2, (0, 255, 0), 2)
            center = (p1[0] + (p2[0] - p1[0]) // 2, p1[1] + (p2[1] - p1[1]) // 2)
            object[2].append(center)
            object[1].update(np.array([[center[0]], [center[1]]]))

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        print("Failed to read the video")
        return
    
    objects = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        update_trackers(objects, frame)
        cv2.imshow("Frame", frame)

        key = cv2.waitKey(30) & 0xFF
        if key == ord('p'):
            selected_objects = select_objects(frame)
            objects.extend(selected_objects)
            track_objects(frame, selected_objects)
        elif key == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "Cyclist and vehicle Tracking - 2.mp4"
    main(video_path)


## Click on the Thumbnail it will open sample video on my youtube chanal. I wanted to put the video here but since storage is limited I can not uploaded on Git.

[![Video Title](1.PNG)](https://youtu.be/bZ4jAOuXk0Y "Video Title - Click to Watch!")
[![Video Title](2.PNG)](https://youtu.be/bZ4jAOuXk0Y "Video Title - Click to Watch!")
[![Video Title](3.PNG)](https://youtu.be/RuykFveCeaE "Video Title - Click to Watch!")


## Task 3: Kalman Filter (50 points)

Use the  [filterpy](https://filterpy.readthedocs.io/en/latest/kalman/KalmanFilter.html) library to implement Kalman filters that will track the cyclist and the vehicle (if present) in the video. You will need to use the detections from the previous task to initialize and run the Kalman filter. 

You need to deliver a video that contains the trajectory of the objects as a line that connects the pixels that the tracker indicated. You can use the `ffmpeg` command line tool and OpenCV to superpose the bounding box of the drone on the video as well as plot its trajectory. 

Suggest methods that you can use to address  false positives and how the tracker can help you in this regard.

You will need to have one Kalman filter to track each of the required and present objects (cyclist and vehicle).
